## Google Hashcode 2018 problem: assign rides to cars
coder:Donal Kelly. This attempt is primitive- it currently randomly assigns rides to cars.

The full problem description (pdf) is available here: https://storage.googleapis.com/coding-competitions.appspot.com/HC/2018/hashcode2018_qualification_task.pdf


read text file

In [25]:

filename = "data_files\\b_should_be_easy.in"
inputfile = open(filename,"r")
header = inputfile.readline().rstrip('\n').split(" ")

rides = []
for line in inputfile:
    rides.append([int(x) for x in line.rstrip('\n').split(" ")])
inputfile.close()
print(str(len(rides)) + " rides ")

300 rides 


**Process Input**

In [26]:

rows = int(header[0])
columns = int(header[1])
vehicle_count = int(header[2])
ride_count = int(header[3])
bonus_value = int(header[4])
timesteps = int(header[5])

#print(rows,columns, vehicle_count, ride_count, bonus_value, timesteps)

#append distance to each ride

for r in rides:
    r.append(abs(r[2]-r[0])+abs(r[3]-r[1]))
    r.append(0)
#print(rides)

#assigned_rides = list.fromkeys(range(vehicle_count),[])




**Assign Rides V1: Random**

In [22]:
import random

#set up assignments
assigned_rides = [[] for x in range(vehicle_count)]

#print(assigned_rides)
for i, r in enumerate(rides):
    #print(i)
    random_v = random.randint(0,vehicle_count-1)
    #print(random_v)
    assigned_rides[random_v].append(i)    
print("assigned all rides randomly")

[[79], [15, 86, 112, 227], [155, 179, 267], [81, 246, 294], [72, 105], [31, 91, 117, 254], [54, 60, 128, 260], [82, 224, 261], [28, 163, 198, 281], [92, 230], [251, 268, 272], [134, 162], [24], [156, 217, 229], [], [171, 172], [7, 12, 40], [57, 74, 123, 132, 293], [3, 8, 140, 213], [41, 284], [130, 196, 220, 245], [61, 113, 165, 282, 290, 299], [49, 83, 89, 177], [168], [48, 115], [21, 216], [36, 64, 221], [59, 98, 121, 143, 164, 167], [44, 97, 133, 184, 188, 207, 218, 265], [38, 80, 250], [13, 66, 77, 169, 205], [139], [30, 136, 212, 223, 244], [39, 106, 178, 225], [34, 151, 208], [69], [2, 94], [22, 242, 257, 280, 288], [20, 108], [47, 159], [73, 211, 243, 285], [131, 187, 240, 277], [0, 5, 9, 180, 214], [63, 87, 102, 247], [29, 55, 70, 219], [51, 145, 158], [206], [144, 258], [110, 249], [99, 222, 234], [146, 173, 255, 292], [4, 103, 107, 175], [78], [189], [42, 157, 191], [11, 18, 65, 266], [90, 100, 141, 154, 182, 194, 200, 269, 289], [116, 190, 193, 296], [], [274, 278], [10, 62,

## Assign Rides V2: Timesteps Loop ##

This version finds and assigns rides per timestep. It has some basic efforts to reduce wasted checks.

In [29]:
#create a list of vehicles, each v is an array [currentRide,currentRideEndTimeStep,currentX,currentY]
#if currentRide = -1 the vehicle does not have a ride and can look for one

def canVehicleDoRide(t,vX, vY, rStartX, rStartY, rDist, rEarliestStart, rLatestFinish ):
    distanceToStart = (abs(rStartX-vX)+abs(rStartY-vY))
    waitTime =  (rEarliestStart - (t + distanceToStart))

    if waitTime < 0: #if wait time is negative you do not wait at all, so set to 0
        waitTime = 0

    eta = (t + distanceToStart + waitTime + rDist)
    if eta > rLatestFinish:
        return -1
    else:
        return eta

vehicles = [[-1,-1,0,0] for x in range(vehicle_count)]

#set up assignments
assigned_rides = [[] for x in range(vehicle_count)]

##add an element to rides to track their completion

#canDo = [[[1] for y in range(rides)] for x in range(vehicle_count)]
canDo = [[1 for i in range(ride_count)] for j in range(vehicle_count)]
#print(canDo[0][0])


for r in rides:
    if len(r) < 8:
        r.append(0);
    else:
        r[7] = 0;
    if len(r) < 9: #also need to check if a ride is eliminated because it cannot be done
        r.append(0);
    else:
        r[8] = 0;
    
print(rides)

for t in range(timesteps):
    if t % 1000 == 0:
        print("Timestep = " + str(t));
        
    
    #first check if there are any more rides to assign at all - 
    #first remove any rides that can no longer be done
    for rc in rides:
        if r[5] < t:
            rc[8] = 1 #this ride can no longer be done
            
    unassignedRideExists = False
    for rc in rides:
        if rc[8] == 0 and rc[7] == 0:
            unassignedRideExists =  True;
            break;
    
    if unassignedRideExists == False:
        print("All rides assigned or undoable, timestep " + str(t));
        break;
        
    #for each timestep (with rides left to assign) look at each vehicle
    for  i, v,  in enumerate(vehicles):
        #check to see if a ride finishes this timestep
        if v[1] == t:
            #print("Ride " + str(v[0]) + " on vehicle " + str(i) + " finishes now (timestep " + str(t) + ")")
            v[0] = -1
            
        if v[0] == -1:
            #print("vehicle " + str(i) + " can look for a ride")
            #loop through rides to find an available one
            for k, r in enumerate(rides):
                if r[8]== 0 and r[7] == 0 and canDo[i][k] == 1:
                    #this ride has not been assigned so could be assigned
                    #can this car actually do this ride?
                    canDoNow = canVehicleDoRide(t, v[2], v[3], r[0], r[1], r[6], r[4], r[5] )
                   # print("canDoNow = " + str(canDoNow))
                    if canDoNow > 0:
                        r[7] = 1
                        v[0] = k
                        v[1] = canDoNow
                        assigned_rides[i].append(k)   
                        canDo[i][k] = 0 #not sure if this is needed, but vehicle can only do a ride once
                        #print("Assigned ride " + str(k) + " to vehicle "+ str(i) )
                        break
                    else:
                        #this vehicle will never be able o do this ride so don't check again
                        canDo[i][k] = 0
                        
#count the rides that were assigned- were some not done?
totalAssigned = 0
for r in rides:
    totalAssigned+= r[7]
    
print(str(totalAssigned) + "/" + str(len(rides)) + " Rides Assigned" )
            

[[395, 43, 296, 955, 244, 3620, 1011, 0, 0], [734, 467, 300, 833, 21919, 23595, 800, 0, 0], [473, 815, 21, 810, 24428, 24959, 457, 0, 0], [210, 150, 387, 12, 12646, 13491, 315, 0, 0], [314, 563, 141, 419, 2744, 3381, 317, 0, 0], [237, 432, 151, 805, 16506, 17940, 459, 0, 0], [264, 491, 445, 767, 14518, 16078, 457, 0, 0], [219, 215, 154, 449, 2806, 3698, 299, 0, 0], [508, 681, 180, 701, 2744, 3179, 348, 0, 0], [460, 585, 320, 715, 23246, 23518, 270, 0, 0], [732, 600, 101, 985, 15613, 19024, 1016, 0, 0], [790, 935, 686, 121, 12617, 14630, 918, 0, 0], [761, 85, 111, 567, 18517, 21011, 1132, 0, 0], [126, 185, 470, 739, 4056, 5036, 898, 0, 0], [31, 325, 197, 802, 11368, 12202, 643, 0, 0], [190, 614, 588, 972, 5453, 7904, 756, 0, 0], [493, 783, 48, 358, 16842, 18899, 870, 0, 0], [648, 494, 602, 487, 20154, 20301, 53, 0, 0], [108, 973, 280, 646, 6209, 7780, 499, 0, 0], [81, 766, 536, 544, 7102, 7835, 677, 0, 0], [547, 716, 490, 315, 17725, 18263, 458, 0, 0], [498, 128, 223, 589, 8084, 9741, 7

Timestep = 5000
Timestep = 6000
Timestep = 7000
Timestep = 8000
Timestep = 9000
Timestep = 10000
Timestep = 11000
Timestep = 12000
Timestep = 13000
Timestep = 14000
Timestep = 15000
Timestep = 16000
Timestep = 17000
All rides assigned or undoable, timestep 17713
203/300 Rides Assigned


**Generate output**

In [30]:
outputfile = open("HASHCODE2018attemptv1.txt","w")
for key, value in enumerate(assigned_rides):
    s = "" + str(key)
    #print(value)
    for v in value:
        s+= " " + str(v)
    #print(s)
    outputfile.write(s+"\n")
outputfile.close()
    

Work out the score submitted

In [31]:
score = 0
for key, value in enumerate(assigned_rides):
    location = [0,0] #all vehicles start at 0, each ride will leave them at a different point
    timeCounter = 0
    vehicleScore = 0   
    for v in value:
        rideScore = 0
        ride = rides[v]
        startX = ride[0]
        startY = ride[1]
        endX = ride[2]
        endY = ride[3]
        earliestStart = ride[4]
        latestFinish=ride[5]
        distance = ride[6]
            
        distanceToStart = (abs(startX-location[0])+abs(startY-location[1]))
        waitTime =  (earliestStart - (timeCounter + distanceToStart))
        if waitTime < 0: #if wait time is negative you do not wait at all, so set to 0
            waitTime = 0
       
        #can you actually do the ride at all?
        EstimatedTimeFinished = (timeCounter + distanceToStart + waitTime + distance)
        canFinish = (EstimatedTimeFinished <= latestFinish) # True or False
        
        #if you can finish it, add the score and move the location and time on
        bonus = 0
        if canFinish:
            #do they get a bonus?           
            if (timeCounter + distanceToStart <= earliestStart):
                bonus=bonus_value
            #print("Vehicle ",key , ", Ride ",key, " from (",ride[0],",",ride[1],") to (",ride[2]," to ", ride[3],"),")
            #print("distanceToStart:",distanceToStart, "earliestStart:",earliestStart,"waitTime:",waitTime,", distance:",distance)
            #print("EstimatedTimeFinished: ",EstimatedTimeFinished,", latestFinish:",latestFinish,", canFinish:",canFinish,", bonus:",bonus)  
            rideScore += distance + bonus
            score+=rideScore
            #print("rideScore = " , rideScore)
            vehicleScore += rideScore                       
            location = [endX,endY]
            timeCounter += distanceToStart + waitTime + distance 
            #print("Adjusted location: (",endX, ",",endY,"), timeCounter:",timeCounter)
            
    #print("Vehicle Score = ", vehicleScore)
            
print("Total Score = " + str(score))

Total Score = 121247
